In [ ]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline
import os 


In [ ]:
train_path = 
test_path = 
save_path = 

In [ ]:
df = pd.read_csv(train_path)

In [ ]:
df.head()

In [ ]:
df['label_id'] = df['Label'].factorize()[0]

#view the first 10 entries of label_id
df['label_id'][0:10]

df.head(20)

In [ ]:
label_id_df = df[['Label','label_id']].drop_duplicates().sort_values('label_id')

label_id_df

In [ ]:
label_to_id = dict(label_id_df.values)
id_to_label = dict(label_id_df[['label_id','Label']].values)
id_to_label

In [ ]:
label_to_id

In [ ]:
df.groupby('Label').label_id.count()

In [ ]:
df.groupby('Label').label_id.count().plot.bar(ylim = 0)

In [ ]:
chichewa = ['i', 'ine', 'wanga', 'inenso', 'ife', 'athu',
            'athu', 'tokha', 'inu', 'ndinu','iwe ukhoza',
            'wako','wekha','nokha','iye','wake','iyemwini',
            'icho','ndi','zake','lokha','iwo','awo','iwowo',
            'chiyani','amene', 'uyu', 'uyo', 'awa', "ndili", 
            'ndi', 'ali','anali','khalani','akhala','kukhala',
            ' Khalani nawo','wakhala','anali','chitani',
            'amachita','kuchita', 'a', 'an', 'pulogalamu ya',
            'ndi', 'koma', 'ngati', 'kapena', 'chifukwa',
            'monga', 'mpaka', 'pamene', 'wa', 'pa ',' by',
            'chifukwa' 'ndi','pafupi','kutsutsana','pakati',
            'kupyola','nthawi', 'nthawi','kale','pambuyo',
            'pamwamba', 'pansipa', 'kuti', 'kuchokera',
            'mmwamba', 'pansi', 'mu', 'kunja', 'kuyatsa', 
            'kuchoka', 'kutha', 'kachiwiri', 'kupitilira',
            'kenako',' kamodzi','apa','apo','liti','pati',
            'bwanji','onse','aliyense','onse','aliyense', 
            'ochepa', 'zambiri', 'ambiri', 'ena', 'otero', 
            'ayi', 'kapena', 'osati', 'okha', 'eni', 'omwewo', 
            'kotero',' kuposa','nawonso',' kwambiri','angathe',
            'ndidzatero','basi','musatero', 'musachite',
            ' muyenera', 'muyenera kukhala','tsopano', 'sali', 
            'sindinathe','​​sanachite','satero','analibe', 
            'sanatero','sanachite','sindinatero','ayi','si', 
            'ma', 'sizingatheke','mwina','sayenera', 'osowa',
            'osafunikira', 'shan' , 'nenani', 'sayenera', 'sanali', 
            'anapambana', 'sangachite', 'sanakonde', 'sangatero']

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(sublinear_tf = True,
                        min_df = 5, 
                        norm = 'l2',
                        encoding = 'latin-1',
                        ngram_range = (1,2),
                        stop_words = chichewa)

features = tfidf.fit_transform(df.Text).toarray()

labels = df.label_id

In [ ]:
features.shape

In [ ]:
label_to_id.items()

In [ ]:
sorted(label_to_id.items())

In [ ]:
from sklearn.feature_selection import chi2


N = 4 # We are going to look for the top 4  categories.


#for each Label, find words that are highly correleated to it.

for Label, label_id in sorted(label_to_id.items()):
    #Do chi2 analyses of all items in this category.
    features_chi2 = chi2(features, labels == label_id)
    #sorts the indices of features_chi2[0] - the chi-squared stats of each feature
    indices = np.argsort(features_chi2[0])
    #converts indices to feature names (in increasing order of chi squared stat values)
    feature_names = np.array(tfidf.get_feature_names())[indices]
    #list of single word features (in increasing order of chi squared stat values)
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    #list of two-word features (in increasing order of chi-squared stat values)
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    
    print("# '{}':".format(Label))
    print(" .Most correlated unigrams: \n    .{}".format('\n       .'.join(unigrams[-N:]))) #print 4 unigrams with highest chi squared stat
    print(" . Most correleated bigrams:\n     .{}".format('\n      .'.join(bigrams[-N:]))) # print 4 bigrams with highest chi squared stat

In [ ]:
features_chi2

**This is computationally very expensive use carefully crashes kernels**

In [ ]:
from sklearn.manifold import TSNE

#Sampling a subset of our dataset because t-sne is computationally expensive 
SAMPLE_SIZE = int(len(features) * 0.1)

np.random.seed(0)

#randomly select 10% of the samples
indices = np.random.choice(range(len(features)), size = SAMPLE_SIZE, replace = False)

#Array of all projected features of 10% of chosen samples.
projected_features = TSNE(n_components = 2, random_state = 0).fit_transform(features[indices])

project_features.shape

In [ ]:
my_id = 0 # select a label_id
projected_features[(labels[indices]== my_id).values]

In [ ]:
colors = ['pink', 'green', 'midnightblue', 'orange', 'darkgrey']

#Find points belonging to each Label and plot them 

for label, label_id in sorted(label_to_id.items()):
    points = projected_features[(labels[indices] == category_id).values]
    plt.scatter(points[:,0], point[:,1], s = 30, c = colors[category_id], label = Label)
plt.title("tf-idf feature vector for each article, projected on 2 dimensions",fontdict = dict(fontsize = 15))

plt.legend()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score


models = [
    RandomForestClassifier(n_estimators = 200, max_depth = 100, random_state = 0),
    MultinomialNB(),
    LogisticRegression(random_state = 30),
]

In [ ]:
CV = 5

#Create a dataframe tat will store athe results for all 5 trials
cv_df = pd.DataFrame(index = range(CV * len(models)))
entries = []

In [ ]:
for model in models:
    model_name = model.__class__.__name__
    
    #create 3 models with different 20% test sets and store their results
    accuraciies = cross_val_score(model, features, labels, scoring = 'accuracy', cv = CV)
    
    #Append all 5 accuracies into the entries list
    for fold_idx, accuracy in enumerate(accuracies):
        entries.append((model_name, fold_idx, accuracy))

In [ ]:
cv_df = pd.DataFrame(entries, columns = ['model_name', 'fold_idx','accuracy'])


In [ ]:
import seaborn as sns

sns.boxplot(x = 'model_name', y = 'accuracy', data = cv_df)
sns.stripplot(x = 'model_name', y = 'accuracy', data = cv_df, size = 8, jitter = True, edgecolor = 'gray', linewidth = 2)


In [ ]:
cv_df.groupb('model_name').accuracy.mean()

In [ ]:
cv_df

In [ ]:
from sklearn.model_selection import train_test_split

model = LogisticRegression(random_stat = 0)


#split data
X_train, X_test, y_train,y_test , indices_train , indices_test = train_test_split(features,
                                                                                  labels, 
                                                                                  df.index,
                                                                                  test_size =0.2, 
                                                                                  random_state = 42)

#train the algorithm
model.fit(X_train,y_train)


#make predictions
y_pred_proba  model.predict_proba(X_test)
y_pred  = model.predict(X_test)



In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns


conf_mat = confusion_matrix(y_test, y_pred)
sns.heatmap(conf_mat, annot = True, fmt = 'd',
            xticklabels = label_id_df.Label.values, 
            yticklabels = label_id_df.Label.values)

plt.ylabel('Actual')
plt.xlabel('Predicted')

In [ ]:
from IPython.display import display

for predicted in label_id_df.label_id:
    for actual in label_id_df.label_id:
        if predicted != actual and con_mat[actual, predicted] >= 2:
            print("'{}' predicted as '{}' : {} examples.".format(id_to_label[actual], id_to_label[predicted], conf_mat[actual,predicted]))
            display(df.loc[indices_test[(y_test == actual) & (y_pred == predicted)]]['Text'])
            print('')

In [ ]:
from sklearn.feature_selection import chi2

N = 5
for Label, label_id in sorted(label_to_id.items()):
  indices = np.argsort(model.coef_[label_id])   # This time using the model co-eficients / weights
  feature_names = np.array(tfidf.get_feature_names())[indices]
  unigrams = [v for v in reversed(feature_names) if len(v.split(' ')) == 1][:N]
  bigrams = [v for v in reversed(feature_names) if len(v.split(' ')) == 2][:N]
  print("# '{}':".format(Category))
  print("  . Top unigrams:\n       . {}".format('\n       . '.join(unigrams)))
  print("  . Top bigrams:\n       . {}".format('\n       . '.join(bigrams)))


In [ ]:
texts = [#YOUR SAMPLE TEXTS HERE]
text_features = tfidf.transform(texts)
predictions = model.predict(text_features)
for text, predicted in zip(texts, predictions):
  print('"{}"'.format(text))
  print("  - Predicted as: '{}'".format(id_to_category[predicted]))
  print("")


In [ ]:
test_df = pd.read_csv(test_path)

In [ ]:
test_features = tfidf.transform(test_df.Text.tolist())

Y_pred = model.predict(test_features)

In [ ]:
Y_pred_name = []
for lab_id in Y_pred:
    Y_pred_name.append(id_to_label[lab_id])

In [ ]:
submission = pd.DataFrame([
    "ID":test_df["IDS"]
    "Label":Y_pred_name
])

In [ ]:
submission.to_csv('submission.csv',index = False)